# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`, combining the use of exchange names and locations.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import EdgeLCIA, get_available_methods
import bw2data

One can simply build its own LCIA file.
Let's consider the following LCIA file (saved under `lcia_example_2.json`):

In [2]:
[
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "CH" # now, we want to specify the location of the consumer
    },
    "value": 1.0
  },
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "FR" # now, we want to specify the location of the consumer
    },
    "value": 2.0
  },
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "RER" # now, we want to specify the location of the consumer
    },
    "value": 3.0
  }
]

[{'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'CH'},
  'value': 1.0},
 {'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'FR'},
  'value': 2.0},
 {'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'RER'},
  'value': 3.0}]

In [3]:
# activate the bw project
bw2data.projects.set_current("bw25_ei310")

In [4]:
act = bw2data.Database("ecoinvent-3.10-cutoff").random()
act

'electricity production, hydro, reservoir, alpine region' (kilowatt hour, IN-RJ, None)

In [5]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/Users/romain/anaconda3/envs/bw25/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 342.51it/s]


0.0023886561581069666

In [7]:
LCA.weights

### Generate dataframe of characterization factors used

The `generate_cf_table` method generates a dataframe of the characterization factors used in the calculation. One can see the characterization factors used for each exchange in the system.

In [5]:
df = LCA.generate_cf_table()

In [6]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","sawing, hardwood","sawdust, loose, wet, measured as dry mass",CH,5.394790e-08,1.0,5.394790e-08
1,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","sawdust, wet, measured as dry mass to generic ...","residual softwood, wet",CH,6.792378e-10,1.0,6.792378e-10
2,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","bark chips, wet, measured as dry mass to gener...","residual softwood, wet",CH,7.476226e-10,1.0,7.476226e-10
3,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","planing, lath, hardwood, u=10%","sawnwood, lath, hardwood, dried (u=10%), planed",CH,-8.070241e-12,1.0,-8.070241e-12
4,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","glued laminated timber production, PUR-glue","shavings, softwood, loose, measured as dry mass",CH,2.028429e-07,1.0,2.028429e-07
...,...,...,...,...,...,...,...,...
1205,"Carbon dioxide, from soil or biomass stock","(air, non-urban air or from high stacks)","polycarboxylates production, 40% active substance","polycarboxylates, 40% active substance",RER,7.908900e-09,3.0,2.372670e-08
1206,"Carbon dioxide, from soil or biomass stock","(air, non-urban air or from high stacks)","sodium perborate production, tetrahydrate, powder","sodium perborate, tetrahydrate, powder",RER,3.336929e-13,3.0,1.001079e-12
1207,"Carbon dioxide, from soil or biomass stock","(air, non-urban air or from high stacks)",anthraquinone production,anthraquinone,RER,3.484943e-09,3.0,1.045483e-08
1208,"Carbon dioxide, from soil or biomass stock","(air, non-urban air or from high stacks)","zeolite production, powder","zeolite, powder",RER,1.365865e-08,3.0,4.097596e-08


As expected, only CH, FR and RER-based consumers have been considered in the calculation.

In [7]:
df.groupby("consumer location")["CF"].mean()

consumer location
CH     1.0
FR     2.0
RER    3.0
Name: CF, dtype: float64

But `statistics()` tells us that many exchanges have been ignored.

In [8]:
LCA.statistics()

+----------------------+-----------------------------+
|       Activity       | nitrile-compound production |
|     Method name      |      ('some', 'method')     |
|      Data file       |        lcia_example_2       |
| Unique CFs in method |              3              |
|   Unique CFs used    |              3              |
|  Exc. characterized  |             1210            |
| Exc. uncharacterized |            334699           |
+----------------------+-----------------------------+


We can extend the application of CFs to other exchanges, using the helping functions: `map_aggregate_locations`, `map_disaggregate_location`, `map_dynamic_locations` and `map_remaining_locations_to_global`.

* `map_aggregate_locations` maps exchanges with aggregate locations (e.g., "RER", RLA", etc.) to weighted-average CFs.
* `map_disaggregate_location` maps exchanges with locations that belong to aggregate locations ("CA-QC") to CFs of aggregate locations (e.g., "CA").
* `map_dynamic_locations` maps exchanges with dynamic locations (e.g., "RoW", "RoW") to CFs calculated as the weighted-average of all locations minus those excluded by dynamic locations' perimeter (e.g., "GLO minus CH").
* `map_remaining_locations_to_global` maps exchanges with remaining locations to the global CFs.

In [5]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.map_aggregate_locations()
LCA.map_dynamic_locations()
LCA.map_contained_locations()
LCA.map_remaining_locations_to_global()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/Users/romain/anaconda3/envs/bw25/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 342.15it/s]


Handling static regions...


100%|██████████| 334710/334710 [00:01<00:00, 273099.79it/s]
0it [00:00, ?it/s]


...

Handling unmatched locations...


100%|██████████| 332517/332517 [00:01<00:00, 316420.34it/s]
0it [00:00, ?it/s]


Handling remaining exchanges...


100%|██████████| 331787/331787 [00:02<00:00, 140550.25it/s]
0it [00:00, ?it/s]


0.012765744558138581

In [6]:
LCA.statistics()

+----------------------+-------------------------------------------+
|       Activity       | electricity production, hydro, reservoir, |
|                      |               alpine region               |
|     Method name      |             ('some', 'method')            |
|      Data file       |               lcia_example_2              |
| Unique CFs in method |                     4                     |
|   Unique CFs used    |                     7                     |
|  Exc. characterized  |                    6462                   |
| Exc. uncharacterized |                   329458                  |
+----------------------+-------------------------------------------+


In [7]:
LCA.weights

{'CH': 8921981, 'FR': 66548530, 'RER': 0}

In [8]:
df = LCA.generate_cf_table()

In [9]:
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, fossil","(air, urban air close to ground)",treatment of wastewater from plywood productio...,wastewater from plywood production,CH,1.501831e-14,1.000000,1.501831e-14
1,"Carbon dioxide, fossil","(air, urban air close to ground)","treatment of sewage sludge, 70% water, WWT, WW...","sewage sludge, 70% water, WWT, WW from ceramic...",CH,3.244456e-19,1.000000,3.244456e-19
2,"Carbon dioxide, fossil","(air, urban air close to ground)","treatment of sewage sludge, 70% water, WWT, WW...","sewage sludge, 70% water, WWT, WW from soft fi...",CH,3.915898e-20,1.000000,3.915898e-20
3,"Carbon dioxide, fossil","(air, urban air close to ground)","treatment of waste emulsion paint, hazardous w...",waste emulsion paint,CH,4.369779e-12,1.000000,4.369779e-12
4,"Carbon dioxide, fossil","(air, urban air close to ground)","treatment of refinery sludge, hazardous waste ...",refinery sludge,CH,2.068310e-10,1.000000,2.068310e-10
...,...,...,...,...,...,...,...,...
6457,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","wood cladding production, softwood","residual wood, dry",CA-QC,1.553429e-12,1.881782,2.923214e-12
6458,"Carbon dioxide, from soil or biomass stock","(air, non-urban air or from high stacks)","land use change, annual crop","land use change, annual crop",BR-TO,2.019586e-08,1.881782,3.800420e-08
6459,"Carbon dioxide, fossil","(air, non-urban air or from high stacks)",primary zinc production from concentrate,cadmium sludge from zinc electrolysis,CA-QC,1.024672e-17,1.881782,1.928210e-17
6460,"Carbon dioxide, fossil","(air, non-urban air or from high stacks)","electricity production, natural gas, combined ...","electricity, high voltage",CN-SC,4.033947e-07,1.881782,7.591009e-07


We can check the weights used:

In [11]:
LCA.weights

{'CH': 8921981, 'FR': 66548530, 'RER': 0}

By default, if weights are not provided in the consumer sections of LCIA file, population is used. The other option is GDP. To be specified when calling EdgeLICA().

And we can see the weighted-average CFs used:

In [10]:
df.groupby("consumer location")["CF"].mean()

consumer location
AE        1.881782
AR        1.881782
AT        3.000000
AU        1.881782
AU-NSW    1.881782
            ...   
VE        1.881782
VN        1.881782
WEU       1.881782
ZA        1.881782
ZM        1.881782
Name: CF, Length: 227, dtype: float64

We have pre-generated LCIA files for specific methods (AWARE, ImpactWorld+, etc.).
You can get a list like so:

In [12]:
get_available_methods()

[('AWARE 1.2c', 'Country', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'irri', 'may'),
 ('AWARE 1.2c', 'Country', 'irri', 'nov'),
 ('AWARE 1.2c', 'Country', 'irri', 'oct'),
 ('AWARE 1.2c', 'Country', 'irri', 'sep'),
 ('AWARE 1.2c', 'Country', 'irri', 'yearly'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'non', 'irr

Check the other notebooks to see how to use them.